In [5]:
from transformers import AutoTokenizer, MarianMTModel

src = "fr"  # source language
trg = "en"  # target language

model_name = f"Helsinki-NLP/opus-mt-{src}-{trg}"
model = MarianMTModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to translate text
def translate(text, src_lang, trg_lang):
    model_name = f"Helsinki-NLP/opus-mt-{src_lang}-{trg_lang}"
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    translated = model.generate(**tokenizer(text, return_tensors="pt"))
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Function to handle user input and generate translated responses
def chat():
    print("Welcome to the Multilingual Chatbot!")
    print("Type 'exit' to end the conversation.")
    print("")

    while True:
        user_input = input("You ({lang}): ".format(lang=src))

        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Translate user input to target language
        translated_input = translate([user_input], src, trg)[0]

        # Generate response in target language
        generated_ids = model.generate(**tokenizer([translated_input], return_tensors="pt"))
        translated_response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Translate response back to source language
        translated_response_src = translate([translated_response], trg, src)[0]

        print("User ({src_lang}): {user_input}".format(src_lang=trg, user_input=user_input))
        # Display translated response in source language
        print("Chatbot ({lang}): {response}".format(lang=src, response=translated_response_src))

# Start the chat
chat()


Welcome to the Multilingual Chatbot!
Type 'exit' to end the conversation.

User (en): Où es-tu?
Chatbot (fr): Où es-tu?
User (en): Où es-tu?
Chatbot (fr): Où es-tu?
Chatbot: Goodbye!


## Auto detect

In [ ]:
from transformers import AutoTokenizer, MarianMTModel
import fasttext

# Load the language detection model
lang_detect_model = fasttext.load_model('lid.176.bin')

def detect_language(text):
    # Predict the language of the input text
    prediction = lang_detect_model.predict(text)
    # Extract the language code from the prediction
    lang_code = prediction[0][0].split('_')[0]
    return lang_code

def translate(text, src_lang, trg_lang):
    model_name = f"Helsinki-NLP/opus-mt-{src_lang}-{trg_lang}"
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    batch = tokenizer([text], return_tensors="pt")

    generated_ids = model.generate(**batch)
    translated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return translated_text

def auto_translate(text, trg_lang):
    src_lang = detect_language(text)
    translated_text = translate(text, src_lang, trg_lang)
    return translated_text

# Example usage
text_to_translate = "Where's the bus stop?"
target_language = "fr"  # Target language

translated_text = auto_translate(text_to_translate, target_language)
print(translated_text)


ModuleNotFoundError: No module named 'fasttext'